In [ ]:
import pandas as pd
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive
drive.mount ('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/RuleDistance-BOW/'

Mounted at /content/gdrive


IMPORT OF RULESETS FOR UNDER 40 AND OVER 40 GROUPS


In [ ]:
# Load the rulesets
# OVER 40
RegoleOver = pd.read_csv(base_dir+"over40_shortened_rules.csv", header=None) 
RegoleOver['Set'] = 'Over 40'
#UNDER 40
RegoleUnder = pd.read_csv(base_dir+"under40_shortened_rules.csv", header=None) 
RegoleUnder['Set'] = 'Under 40'




In [ ]:
RegoleUnder

In [ ]:
RegoleOver

RULESETS ARRANGEMENT

In [ ]:
RegoleUnder['Regola'] = RegoleUnder.apply(lambda x: x[0][x[0].find('IF')+3:x[0].find('THEN')-1], axis=1)
RegoleUnder['Class'] = RegoleUnder.apply(lambda x: x[0][x[0].find('"')+1:x[0].find('"')+2], axis=1)
#RegoleUnder.drop(RegoleUnder.iloc[:,0] , axis=1, inplace=True)
RegoleUnder.drop(RegoleUnder.columns[0] , axis=1, inplace=True)

RegoleOver['Regola'] = RegoleOver.apply(lambda x: x[0][x[0].find('IF')+3:x[0].find('THEN')-1], axis=1)
RegoleOver['Class'] = RegoleOver.apply(lambda x: x[0][x[0].find('"')+1:x[0].find('"')+2], axis=1)
#RegoleOver.drop(RegoleOver.iloc[:,0] , axis=1, inplace=True)
RegoleOver.drop(RegoleOver.columns[0] , axis=1, inplace=True)

Regole = pd.concat([RegoleOver, RegoleUnder], ignore_index=True)

Regole

,Set,Regola,Class
0,Over 40,Hip.ACC.Mean > 4.132912 AND Hip.yposture.Mean ...,0
1,Over 40,Hip.ACC.Mean > 4.580647 AND Hip.yposture.Mean ...,0
2,Over 40,Hip.yposture.Mean <= 64.290487 AND Ankle.xpost...,0
3,Over 40,Ankle.xposture.Mean <= -4.430574,0
4,Over 40,Hip.ACC.Mean <= 3.961156,1
5,Over 40,Hip.ACC.Mean <= 5.103555 AND Hip.yposture.Mean...,1
6,Over 40,Hip.ACC.Mean <= 4.609755 AND Wrist.jerk.coeffi...,1
7,Under 40,'leg rotational position sag plane' > 12.17500...,0
8,Under 40,'leg rotational velocity sag plane' > 618.7250...,0
9,Under 40,Hip.jerk.Mean > 15.085842 AND 'back rotation p...,0


In [ ]:
Regole.loc[(Regole.Set=='Under 40') ].count()

Set       9
Regola    9
Class     9
dtype: int64

Convert class values to literal

In [ ]:
def convert_litteral(classe):
  if classe == '1':
    return 'Fatigued'
  else:
    return 'Non Fatigued'

Regole['Class'] = Regole['Class'].apply(convert_litteral)

In [ ]:
Regole

CREATION OF THE MATRIX FOR DISTANCE COMPUTATION

In [ ]:
RegoleFatica = Regole.copy()
RegoleFatica.reset_index(drop=True, inplace=True)
RegoleFatica

,Set,Regola,Class
0,Over 40,Hip.ACC.Mean > 4.132912 AND Hip.yposture.Mean ...,Non Fatigued
1,Over 40,Hip.ACC.Mean > 4.580647 AND Hip.yposture.Mean ...,Non Fatigued
2,Over 40,Hip.yposture.Mean <= 64.290487 AND Ankle.xpost...,Non Fatigued
3,Over 40,Ankle.xposture.Mean <= -4.430574,Non Fatigued
4,Over 40,Hip.ACC.Mean <= 3.961156,Fatigued
5,Over 40,Hip.ACC.Mean <= 5.103555 AND Hip.yposture.Mean...,Fatigued
6,Over 40,Hip.ACC.Mean <= 4.609755 AND Wrist.jerk.coeffi...,Fatigued
7,Under 40,'leg rotational position sag plane' > 12.17500...,Non Fatigued
8,Under 40,'leg rotational velocity sag plane' > 618.7250...,Non Fatigued
9,Under 40,Hip.jerk.Mean > 15.085842 AND 'back rotation p...,Non Fatigued


EXTRACTION OF ALL FEATURE plus OPERATOR ELEMENTS; CREATION OF AN ARRAY WITH A SINGLE OCCURRENCE FOR EACH FEATURE+OPERATOR AND COUNT OF HOW MANY TIME IT APPEARS IN THE RULE. 

In [ ]:
terms = []
for index, value in RegoleFatica.loc[:,'Regola'].items():
    for r in value.split(' AND '):
      if r.find('>')>0:
        terms.append(r[:r.find('>')+2].replace(" ", ""))
      else:
        terms.append(r[:r.find('<')+2].replace(" ", ""))
      #print(f"Value : {value}")
counter = Counter(terms)

In [ ]:
counter 

Counter({"'backrotationpositioninsagplane'<=": 4,
         "'backrotationpositioninsagplane'>": 2,
         "'legrotationalpositionsagplane'<=": 1,
         "'legrotationalpositionsagplane'>": 2,
         "'legrotationalvelocitysagplane'<=": 3,
         "'legrotationalvelocitysagplane'>": 3,
         'Ankle.xposture.Mean<=': 1,
         'Ankle.xposture.Mean>': 2,
         'Hip.ACC.Mean<=': 5,
         'Hip.ACC.Mean>': 3,
         'Hip.jerk.Mean>': 1,
         'Hip.yposture.Mean<=': 3,
         'Hip.yposture.Mean>': 1,
         'Wrist.jerk.coefficient.of.variation<=': 1,
         'Wrist.jerk.coefficient.of.variation>': 1})

In [ ]:
for i in counter:
  RegoleFatica.loc[:,i]= 0
  RegoleFatica[i+'Value']=0.0
  RegoleFatica[i+'ValueNorm']=0.0

#RegoleFatica

BROWSE ALL THE RULES AND PUT A 1 IF FEATURE+OPERATOR IS PRESENT, 0 OTHERWISE; IN THE 'Value' COLUMN, PUT THE RULES THRESHOLD VALUES



In [ ]:
for c in counter:
  for index, value in RegoleFatica.loc[:,'Regola'].items():
    for r in value.split(' AND '):
      if r[:r.find('>')+2].replace(" ", "") == c:
        RegoleFatica.loc[index, c] = 1
        RegoleFatica.loc[index, c+'Value'] = float(r.split('>')[1].strip())
      if r[:r.find('<')+2].replace(" ", "") == c:
        RegoleFatica.loc[index, c] = 1
        RegoleFatica.loc[index, c+'Value'] = float(r.split('<=')[1].strip())

In [ ]:
RegoleFatica

Browse again all the rules and normalize 'Value' column elements according to the following criteria:


*   If feature+operator is not present in the rule: 0
*   If maximum 'Value' for a feature+operator = minimum 'Value' for a feature+operator: 1
*   Otherwise: (value-min(value))/(Max(value)-Min(value))





In [ ]:
for c in counter:
  MAX = RegoleFatica.loc[RegoleFatica[c] == 1, c+'Value'].max()
  MIN = RegoleFatica.loc[RegoleFatica[c] == 1, c+'Value'].min()
  denominatore = MAX-MIN

  for index, value in RegoleFatica.loc[:,c+'Value'].items():
    if RegoleFatica.loc[index, c] == 1:
      if denominatore == 0:
        RegoleFatica.loc[index, c+'ValueNorm'] = 1
      else:  
        RegoleFatica.loc[index, c+'ValueNorm'] = (RegoleFatica.loc[index, c+'Value']-MIN)/denominatore
    else:
      RegoleFatica.loc[index, c+'Value'] = 0

In [ ]:
RegoleFatica

In [ ]:
for c in counter:
  del RegoleFatica[c+'Value']

In [ ]:
RegoleFatica

,Set,Regola,Class,Hip.ACC.Mean>,Hip.ACC.Mean>ValueNorm,Hip.yposture.Mean<=,Hip.yposture.Mean<=ValueNorm,Wrist.jerk.coefficient.of.variation<=,Wrist.jerk.coefficient.of.variation<=ValueNorm,Ankle.xposture.Mean>,Ankle.xposture.Mean>ValueNorm,'backrotationpositioninsagplane'<=,'backrotationpositioninsagplane'<=ValueNorm,Ankle.xposture.Mean<=,Ankle.xposture.Mean<=ValueNorm,Hip.ACC.Mean<=,Hip.ACC.Mean<=ValueNorm,Hip.yposture.Mean>,Hip.yposture.Mean>ValueNorm,Wrist.jerk.coefficient.of.variation>,Wrist.jerk.coefficient.of.variation>ValueNorm,'legrotationalpositionsagplane'>,'legrotationalpositionsagplane'>ValueNorm,'legrotationalvelocitysagplane'>,'legrotationalvelocitysagplane'>ValueNorm,Hip.jerk.Mean>,Hip.jerk.Mean>ValueNorm,'legrotationalvelocitysagplane'<=,'legrotationalvelocitysagplane'<=ValueNorm,'backrotationpositioninsagplane'>,'backrotationpositioninsagplane'>ValueNorm,'legrotationalpositionsagplane'<=,'legrotationalpositionsagplane'<=ValueNorm
0,Over 40,Hip.ACC.Mean > 4.132912 AND Hip.yposture.Mean ...,Non Fatigued,1,0.403512,1,1.0,1,1.0,0,0.0,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.0
1,Over 40,Hip.ACC.Mean > 4.580647 AND Hip.yposture.Mean ...,Non Fatigued,1,1.000000,1,1.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.0
2,Over 40,Hip.yposture.Mean <= 64.290487 AND Ankle.xpost...,Non Fatigued,0,0.000000,1,0.0,0,0.0,1,1.0,1,0.012048,0,0.0,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.0
3,Over 40,Ankle.xposture.Mean <= -4.430574,Non Fatigued,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.000000,1,1.0,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.0
4,Over 40,Hip.ACC.Mean <= 3.961156,Fatigued,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,1,0.164182,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.0
5,Over 40,Hip.ACC.Mean <= 5.103555 AND Hip.yposture.Mean...,Fatigued,0,0.000000,0,0.0,0,0.0,1,0.0,0,0.000000,0,0.0,1,1.000000,1,1.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.0
6,Over 40,Hip.ACC.Mean <= 4.609755 AND Wrist.jerk.coeffi...,Fatigued,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,1,0.638719,0,0.0,1,1.0,0,0.0,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.0
7,Under 40,'leg rotational position sag plane' > 12.17500...,Non Fatigued,0,0.000000,0,0.0,0,0.0,0,0.0,1,0.689759,0,0.0,0,0.000000,0,0.0,0,0.0,1,0.0,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.0
8,Under 40,'leg rotational velocity sag plane' > 618.7250...,Non Fatigued,0,0.000000,0,0.0,0,0.0,0,0.0,1,1.000000,0,0.0,0,0.000000,0,0.0,0,0.0,0,0.0,1,0.852422,0,0.0,0,0.000000,0,0.0,0,0.0
9,Under 40,Hip.jerk.Mean > 15.085842 AND 'back rotation p...,Non Fatigued,0,0.000000,0,0.0,0,0.0,0,0.0,1,0.000000,0,0.0,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.000000,1,1.0,0,0.000000,0,0.0,0,0.0


In [ ]:
RegoleFatica.to_excel(base_dir+'BagOfWords_fatica_ristretto.xlsx', index=True)

CALCOLO DISTANZA TRA VETTORI

Prima i valori vanno trasformati in array

In [ ]:
matrice = RegoleFatica.iloc[:,3:RegoleFatica.shape[1]].to_numpy()
matrice

array([[1.        , 0.4035123 , 1.        , 1.        , 1.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [1.        , 1.        , 1.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 1.        , 1.        , 1.        , 0.01204819,
        0.        , 0.        , 0.        , 0.

In [ ]:
matrice.shape

(16, 30)

Calcolo la vicinanza delle regole passando la matrice che ho costruito alla funzione cosine_similarity

In [ ]:
Vicinanza = cosine_similarity (matrice, matrice)

In [ ]:
MatriceVicinanza = pd.DataFrame(Vicinanza,
                   index = RegoleFatica['Set'] + ' - ' + RegoleFatica['Class'] + ' - ' + RegoleFatica['Regola'],
                   #index=phrases,
                   columns = RegoleFatica['Set'] + ' - ' + RegoleFatica['Class'] + ' - ' + RegoleFatica['Regola'])
                   #columns=phrases)

MatriceVicinanza.head()

,Over 40 - Non Fatigued - Hip.ACC.Mean > 4.132912 AND Hip.yposture.Mean <= 64.906740 AND Wrist.jerk.coefficient.of.variation <= 116.520000,Over 40 - Non Fatigued - Hip.ACC.Mean > 4.580647 AND Hip.yposture.Mean <= 64.906740,Over 40 - Non Fatigued - Hip.yposture.Mean <= 64.290487 AND Ankle.xposture.Mean > -1.778758 AND 'back rotation position in sag plane' <= 7.780000,Over 40 - Non Fatigued - Ankle.xposture.Mean <= -4.430574,Over 40 - Fatigued - Hip.ACC.Mean <= 3.961156,Over 40 - Fatigued - Hip.ACC.Mean <= 5.103555 AND Hip.yposture.Mean > 64.290487 AND Ankle.xposture.Mean > -5.555916,Over 40 - Fatigued - Hip.ACC.Mean <= 4.609755 AND Wrist.jerk.coefficient.of.variation > 107.135000,Under 40 - Non Fatigued - 'leg rotational position sag plane' > 12.175000 AND 'back rotation position in sag plane' <= 8.905000,Under 40 - Non Fatigued - 'leg rotational velocity sag plane' > 618.725000 AND 'back rotation position in sag plane' <= 9.420000,Under 40 - Non Fatigued - Hip.jerk.Mean > 15.085842 AND 'back rotation position in sag plane' <= 7.760000,Under 40 - Non Fatigued - 'leg rotational velocity sag plane' > 670.460000,Under 40 - Non Fatigued - 'leg rotational velocity sag plane' > 319.900000 AND 'leg rotational position sag plane' > 28.865000,Under 40 - Fatigued - 'leg rotational velocity sag plane' <= 672.835000 AND 'back rotation position in sag plane' > 9.185000,Under 40 - Fatigued - Hip.ACC.Mean <= 3.736752 AND 'leg rotational velocity sag plane' <= 598.105000,Under 40 - Fatigued - Hip.ACC.Mean <= 4.658695 AND 'leg rotational velocity sag plane' <= 636.400000 AND 'leg rotational position sag plane' <= 12.840000,Under 40 - Fatigued - Hip.ACC.Mean > 3.830028 AND 'back rotation position in sag plane' > 9.225000
Over 40 - Non Fatigued - Hip.ACC.Mean > 4.132912 AND Hip.yposture.Mean <= 64.906740 AND Wrist.jerk.coefficient.of.variation <= 116.520000,1.000000,0.748952,0.220049,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.254095
Over 40 - Non Fatigued - Hip.ACC.Mean > 4.580647 AND Hip.yposture.Mean <= 64.906740,0.748952,1.000000,0.249995,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.288675
Over 40 - Non Fatigued - Hip.yposture.Mean <= 64.290487 AND Ankle.xposture.Mean > -1.778758 AND 'back rotation position in sag plane' <= 7.780000,0.220049,0.249995,1.000000,0.0,0.0,0.223603,0.00000,0.320407,0.262123,0.28867,0.0,0.0,0.0,0.000000,0.000000,0.000000
Over 40 - Non Fatigued - Ankle.xposture.Mean <= -4.430574,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.00000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000,0.000000
Over 40 - Fatigued - Hip.ACC.Mean <= 3.961156,0.000000,0.000000,0.000000,0.0,1.0,0.513760,0.59059,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.697765,0.504636,0.000000


In [ ]:
MatriceVicinanza.to_excel(base_dir+'MatriceVicinanza_fatica_corretto.xlsx', index=True)